In [46]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import MultiLineString, LineString
import random
import simpy
import numpy as np

In [10]:
highways_gdf = gpd.read_file('New folder/NTAD_National_Highway_Freight_Network_-633960569793832517.gpkg', layer='National_Highway_Freight_Network__NHFN_')
highways_gdf = highways_gdf.to_crs(epsg=3857)
stops_gdf = gpd.read_file('New folder\Truck_Stop_Parking.gpkg', layer = 'Truck_Stop_Parking')
stops_gdf = stops_gdf.to_crs(epsg=3857)

selected_segments = gpd.read_file("New folder/Segments.gpkg")
recommended_segments = gpd.read_file("New folder/recommended_segments.gpkg")
shortest_segmenets = gpd.read_file("New folder/shortest_segments.gpkg")

selected_columns = ['Class', 'Class_Description', 'Road_Name', 'FAF5 Domestic Truck Flows by Commodity_2050Base_TOT Trips_50Base Dom', 'geometry']  # Replace with the actual column names
new_df = selected_segments[selected_columns]
new_shortest = shortest_segmenets[selected_columns]
new_recommended = recommended_segments[selected_columns]


directions = gpd.read_file("New folder/directions.gpkg")
shortest_directions = gpd.read_file("New folder/shortest_route.gpkg")
recommended_directions = gpd.read_file("New folder/reccomended_route.gpkg")
stops = gpd.read_file("New folder\Truck_Stop_Parking.gpkg")

directions = directions.to_crs(epsg=3857)
shortest_directions = shortest_directions.to_crs(epsg=3857)
recommended_directions = recommended_directions.to_crs(epsg=3857)

buffer_distance = 1609.34 
directions_buffer = directions.copy()
shortest_buffer = shortest_directions.copy()
recommended_buffer = recommended_directions.copy()
directions_buffer['geometry'] = directions_buffer.geometry.buffer(buffer_distance)
shortest_buffer['geometry'] = shortest_buffer.geometry.buffer(buffer_distance)
recommended_buffer['geometry'] = recommended_buffer.geometry.buffer(buffer_distance)
directions_combined = directions_buffer.unary_union
shortest_combined = shortest_buffer.unary_union
recommended_combined = recommended_buffer.unary_union

new_df = new_df.to_crs(epsg=4326)
new_shortest = new_shortest.to_crs(epsg=4326)
new_recommended = new_recommended.to_crs(epsg=4326)

filtered_stops_gdf = stops_gdf[stops_gdf.intersects(directions_combined)].to_crs(epsg=4326)
filtered_stops_shortest = stops_gdf[stops_gdf.intersects(shortest_combined)].to_crs(epsg=4326)
filtered_stops_recommended = stops_gdf[stops_gdf.intersects(recommended_combined)].to_crs(epsg=4326)

directions = directions.to_crs(epsg=4326)
shortest_directions = shortest_directions.to_crs(epsg=4326)
recommended_directions = recommended_directions.to_crs(epsg=4326)
# Load your GeoDataFrame (replace with your own data)
gdf_projected = selected_segments.to_crs(epsg=3857)
centroid = gdf_projected.geometry.centroid
m = folium.Map(location=[centroid.to_crs(epsg=4326).y.mean(), centroid.to_crs(epsg=4326).x.mean()], zoom_start=10)
to_plot = selected_segments.iloc[1347]
###STOPS####
geom = filtered_stops_gdf['geometry']
for point in geom:
    lon,lat = point.coords[0]
    folium.CircleMarker(location=[lat, lon], color = 'black').add_to(m)
geom = filtered_stops_shortest['geometry']
for point in geom:
    lon,lat = point.coords[0]
    folium.CircleMarker(location=[lat, lon], color = 'black').add_to(m)   
geom = filtered_stops_recommended['geometry']
for point in geom:
    lon,lat = point.coords[0]
    folium.CircleMarker(location=[lat, lon], color = 'black').add_to(m) 


####ROUTES#####
geom = directions['geometry']
for line in geom:
    folium.PolyLine(locations=[(point[1], point[0]) for point in line.coords], color='rgba(255, 0, 0, 0.6)').add_to(m)
geom = shortest_directions['geometry']
for line in geom:
    folium.PolyLine(locations=[(point[1], point[0]) for point in line.coords], color='rgba(0, 255, 0, 0.6)').add_to(m)   
geom = recommended_directions['geometry']
for line in geom:
    folium.PolyLine(locations=[(point[1], point[0]) for point in line.coords], color='rgba(0, 0, 255, 0.6)').add_to(m)

m

In [244]:
def generate_stations(csv, stops):
    df = pd.read_csv(csv)
    truck_data = []
    for _, row in stops.iterrows():
        name = row['nhs_rest_stop']
        latitude = row['latitude']
        longitude = row['longitude']
        spots = row['number_of_spots']
        geometry = row['geometry']
        
        # Filter rows with matching coordinates and spots
        matching_row = df[(df['latitude'] == latitude) & (df['longitude'] == longitude) & (df['total_spots'] == spots)]
        
    # Handle empty matching_row
        if matching_row.empty:
            continue
    
        # Extract the values from matching_row (assuming there is only one row)
        charging_spots = matching_row.iloc[0]['charging_spots']
        megachargers = matching_row.iloc[0]['megachargers (750kW)']
        mwcs = matching_row.iloc[0]['MWCS (1 MW)']
        slow_chargers = matching_row.iloc[0]['slow_chargers (50kW)']
        station_capacity = matching_row.iloc[0]['station_capacity (MW)']
        
        # Create a buffer of 1-mile radius around the point
        buffer = geometry.buffer((1/59)/2)  # Radius converted to degrees
        # Select nearby roads
        nearby_road = new_df[new_df.intersects(buffer)]
        # Handle the case when no nearby road segments are found
        if nearby_road.empty:
            print("empty")
            print(nearby_road)
            print(row)
            total_trucks = 0
        else:
            total_trucks = int(nearby_road['FAF5 Domestic Truck Flows by Commodity_2050Base_TOT Trips_50Base Dom'].mean())
        
        # Create the entry to store in truck_data list
        entry = {
            'Name': name,
            'Latitude': latitude,
            'Longitude': longitude,
            'Spots': spots,
            'Charging_Spots': charging_spots,
            'Megachargers': megachargers,
            'MWCS': mwcs,
            "slow chargers": slow_chargers,
            "Station Capacity": station_capacity,
            "Daily Truck Flow": total_trucks,
            "geometry": geometry
        }
        
        truck_data.append(entry)
    truck_data = gpd.GeoDataFrame(truck_data, crs='EPSG:4326')
    return truck_data

shortest_stops_gdf = generate_stations("truck_stops_electrification.csv",filtered_stops_shortest )
fastest_stops_gdf = generate_stations("truck_stops_electrification.csv",filtered_stops_gdf )
recommended_stops_gdf = generate_stations("truck_stops_electrification.csv",filtered_stops_recommended )

In [15]:
shortest_stops_gdf = shortest_stops_gdf.drop('slow chargers', axis = 1)
fastest_stops_gdf = fastest_stops_gdf.drop('slow chargers', axis = 1)
recommended_stops_gdf = recommended_stops_gdf.drop('slow chargers', axis = 1)

In [241]:
def station_metrics(data, interval_length = 30, sim_time = 2880):
    df = pd.DataFrame(data.customer_data)
    intervals = range(0, sim_time + 1, interval_length)
    average_wait_times = []
    average_capacity = []
    turn_away_times = data.trucks_turned_away
    for start_time in intervals:
        # Filter customers who arrived during the current 15-minute interval
        interval_data = df[(df['Joined Queue at'] >= start_time) & (df['Joined Queue at'] < start_time + interval_length)]
        trucks_turned_away_in_interval = any(t >= start_time and t < start_time + interval_length for t in turn_away_times)

        if not interval_data.empty:
            avg_wait_time = interval_data['Queue Wait Time'].mean().round(2)
            station_capacity = interval_data['Station Capacity (%)'].mean().round(2)
        elif trucks_turned_away_in_interval:
            avg_wait_time = "Station Full"
            station_capacity = 100
        else:
            avg_wait_time = "Station Empty"
            station_capacity = 0
        average_wait_times.append({
            'Interval Start': f"{start_time}-{start_time + interval_length}",
            'Average Wait Time': avg_wait_time,
            'Average Station Capacity': station_capacity,
            
        })

    # Outputting the results
    avg_wait_times_df = pd.DataFrame(average_wait_times)
    def minutes_to_time(minutes):
        hours = minutes // 60
        mins = minutes % 60
        return f"{hours:02}:{mins:02}"


    interval_length = 30  # 30-minute intervals
    avg_wait_times_df['Interval Start'] = avg_wait_times_df['Interval Start'].apply(lambda x: x.split('-'))  # Split the interval to start and end times
    avg_wait_times_df['Interval Start'] = avg_wait_times_df['Interval Start'].apply(
        lambda x: f"{minutes_to_time(int(x[0]))} - {minutes_to_time(int(x[1]))}"
    )
    return avg_wait_times_df

def get_info(stops):
    station_metrics_dict = {}
    for _, station in stops.iterrows():
        num_stations = station['Charging_Spots']
        num_spots = station['Spots'] - num_stations
        daily_flow = station['Daily Truck Flow']
        q = ChargingStation(num_stations=2, num_spots=10,  daily_flow=daily_flow, sim_time=2880)
        q.env.process(q.run_station_sim())
        q.env.run()
        station_key = (station['Name'], _)
        station_metrics_dict[station_key] = station_metrics(q)
    return station_metrics_dict

class ChargingStation:
    def __init__(self, num_stations, num_spots, daily_flow, sim_time):
        self.env = simpy.Environment()
        self.total_spots = num_spots + num_stations
        self.stations = simpy.Resource(self.env, num_stations)
        self.queue_spots = num_spots
        self.customer_data = []
        self.trucks_turned_away = []
        self.daily_flow = daily_flow * 0.4
        self.station_volume_count = 0
        self.i = 1
        self.sim_time = sim_time
        self.morning_rate = ((self.daily_flow * 0.25) / (3 * 60)) * 0.05 #6AM - 9AM
        self.midday_rate = ((self.daily_flow * 0.40) / (6 * 60)) * 0.05 #9AM - 3PM
        self.evening_rate = ((self.daily_flow * 0.25) / (3 * 60) ) * 0.05 #3PM - 6PM
        self.overnight_rate = ((self.daily_flow * 0.10) / (12 *60)) * 0.05#6PM - 6AM
        
    def charging(self, customer):
        # random_time = max(1, np.random.normal(5, 4))
        service_time = np.random.triangular(20, 40, 70)
        yield self.env.timeout(service_time)
        
    def truck(self, name):

        if len(self.stations.queue) == self.queue_spots:
            self.trucks_turned_away.append(self.env.now)
            return 
        

        queue = self.env.now
        queue_pos = len(self.stations.queue) + 1

            
        self.i += 1
        with self.stations.request() as request:
            yield request
            start_service = self.env.now
            yield self.env.process(self.charging(name))
            end_service = self.env.now
            
            self.customer_data.append({
            'Customer ID': name,
            'Joined Queue at': queue,
            'Queue Position': queue_pos,
            'Station Capacity (%)': ((len(self.stations.users) + len(self.stations.queue)) / (self.total_spots)) * 100,
            'Start Service Time': start_service,
            'End Service Time': end_service,
            'Service Time': end_service - start_service,
            'Queue Wait Time': start_service - queue,
            'Total Process Time': end_service - queue
            })
            
    def run_station_sim(self):
        while True:
            current_hour = (self.env.now // 60) % 24
            if 6 <= current_hour <= 9:
                arrival_rate = self.morning_rate
            elif 9 <= current_hour <= 15:
                arrival_rate = self.midday_rate
            elif 15 <= current_hour < 18:
                arrival_rate = self.evening_rate
            else:
                arrival_rate = self.overnight_rate
            yield self.env.timeout(random.expovariate(arrival_rate))
            
            
            current_time = self.env.now
            if current_time <= self.sim_time:
                self.env.process(self.truck(self.i))
            else:
                break
            

In [248]:
shortest_station_metrics_dict = get_info(shortest_stops_gdf)
fastest_station_metrics_dict = get_info(fastest_stops_gdf)


In [253]:

random_df_key = random.choice(list(fastest_station_metrics_dict.keys()))
random_df = fastest_station_metrics_dict[random_df_key]

# Step 2: Select a random row and column from the chosen DataFrame
random_row = random.randint(0, len(random_df) - 1)  # Random row index
random_col = random.choice(random_df.columns)  # Random column name

# Get the random value
random_df

,Interval Start,Average Wait Time,Average Station Capacity
0,00:00 - 00:30,Station Empty,0.0
1,00:30 - 01:00,Station Empty,0.0
2,01:00 - 01:30,Station Empty,0.0
3,01:30 - 02:00,Station Empty,0.0
4,02:00 - 02:30,Station Empty,0.0
...,...,...,...
92,46:00 - 46:30,Station Empty,0.0
93,46:30 - 47:00,Station Empty,0.0
94,47:00 - 47:30,Station Empty,0.0
95,47:30 - 48:00,Station Empty,0.0


In [269]:
random_df_key = random.choice(list(fastest_station_metrics_dict.keys()))
random_df = fastest_station_metrics_dict[random_df_key]
random_df['Start Hour'] = random_df['Interval Start'].apply(lambda x: int(x.split(":")[0]))

# Filter rows where Start Hour is between 6 (6:00 AM) and 10 (10:00 AM)
filtered_df = random_df[(random_df['Start Hour'] >= 6) & (random_df['Start Hour'] < 10)]

# Drop 'Start Hour' column for cleaner output if not needed
filtered_df = filtered_df.drop(columns=['Start Hour'])
filtered_df

,Interval Start,Average Wait Time,Average Station Capacity
12,06:00 - 06:30,Station Empty,0.00
13,06:30 - 07:00,4.82,25.00
14,07:00 - 07:30,0.0,25.00
15,07:30 - 08:00,4.93,54.17
16,08:00 - 08:30,33.76,80.56
17,08:30 - 09:00,76.02,95.83
18,09:00 - 09:30,161.92,96.67
19,09:30 - 10:00,204.51,100.00
